In [ ]:
!pip install -q pyomo

     |████████████████████████████████| 9.2 MB 3.8 MB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 


In [ ]:
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [ ]:
from pyomo.environ import *
import numpy as np
from math import pi
import pandas as pd

# Test Data

In [ ]:
Sbase = 10 # MW

buses = {
    1: [1, 0, 1.00, 0.0, 0.0, 0.0, 0.0, 0.0],
    2: [2, 1, 1.01, 0.0, 0.0, 0.0, 0.0, 0.0],
    3: [3, 2, 1.00, 0.0, 0.0, 0.0, 0.3, 0.1]
}

lines = {
    1: [1, 2, 0.0192, 0.0575, 0.0264, 1, 30/Sbase],
    #2: [1, 3, 0.0452, 0.1852, 0.0204, 1, 0.3/Sbase],
    3: [2, 3, 0.0570, 0.1737, 0.0184, 1, 1/Sbase]
}

gens = {
    1: [1, 0/Sbase, 20/Sbase, -20/Sbase, 100/Sbase, 0.00375, 2, 0],
    2: [2, 0/Sbase, 20/Sbase, -20/Sbase, 100/Sbase, 0.0175, 1.75, 0]
}

# IEEE30 BUS

In [ ]:
Sbase = 100
buses_arr = pd.read_excel('IEEE30.xlsx', 'BUS').to_numpy().astype('float64')
lines_arr = pd.read_excel('IEEE30.xlsx', 'RAMAS').to_numpy().astype('float64')
gens_arr = pd.read_excel('IEEE30.xlsx', 'GEN').to_numpy().astype('float64')

# Pre-processing

In [ ]:
buses = dict()
lines = dict()
gens = dict()
for i, row in enumerate(buses_arr):
  buses[i+1]= list(row)

for i, row in enumerate(lines_arr):
  row[6] = 100
  lines[i+1] = list(row)

for i, row in enumerate(gens_arr):
  gens[i+1] = list(row)



In [ ]:
lines

{1: [1.0, 2.0, 0.0192, 0.0575, 0.0264, 1.0, 100.0],
 2: [1.0, 3.0, 0.0452, 0.1852, 0.0204, 1.0, 100.0],
 3: [2.0, 4.0, 0.057, 0.1737, 0.0184, 1.0, 100.0],
 4: [3.0, 4.0, 0.0132, 0.0379, 0.0042, 1.0, 100.0],
 5: [2.0, 5.0, 0.0472, 0.1983, 0.0209, 1.0, 100.0],
 6: [2.0, 6.0, 0.0581, 0.1763, 0.0187, 1.0, 100.0],
 7: [4.0, 6.0, 0.0119, 0.0414, 0.0045, 1.0, 100.0],
 8: [5.0, 7.0, 0.046, 0.116, 0.0102, 1.0, 100.0],
 9: [6.0, 7.0, 0.0267, 0.082, 0.0085, 1.0, 100.0],
 10: [6.0, 8.0, 0.012, 0.042, 0.0045, 1.0, 100.0],
 11: [6.0, 9.0, 0.0, 0.208, 0.0, 1.0155, 100.0],
 12: [6.0, 10.0, 0.0, 0.556, 0.0, 0.9629, 100.0],
 13: [9.0, 11.0, 0.0, 0.208, 0.0, 1.0, 100.0],
 14: [9.0, 10.0, 0.0, 0.11, 0.0, 1.0, 100.0],
 15: [4.0, 12.0, 0.0, 0.256, 0.0, 1.0, 100.0],
 16: [12.0, 13.0, 0.0, 0.14, 0.0, 1.0, 100.0],
 17: [12.0, 14.0, 0.1231, 0.2559, 0.0, 1.0, 100.0],
 18: [12.0, 15.0, 0.0662, 0.1304, 0.0, 1.0, 100.0],
 19: [12.0, 16.0, 0.0945, 0.1987, 0.0, 1.0, 100.0],
 20: [14.0, 15.0, 0.221, 0.1997, 0.0, 1.0, 

# Create Ybus

In [ ]:
nb = len(buses)
nl = len(lines)
ng = len(gens)

Ybus = np.zeros((nb, nb), dtype=np.complex128)
g = np.zeros((nb, nb))
b = np.zeros((nb,nb))
# Loop through lines
for lineid, linedata in lines.items():
  i = int(linedata[0])-1
  k = int(linedata[1])-1
  Z = linedata[2] + 1j*linedata[3]
  Bs = 1j*linedata[4]
  a = linedata[5]

  Ybus[i][i] += (1/(Z*a**2))
  Ybus[k][k] += (1/(Z*a**2))

  Ybus[i][i] += Bs
  Ybus[k][k] += Bs

  Ybus[i][k] -= 1/(a*Z)
  Ybus[k][i] -= 1/(a*Z)

  b[i][k] = Bs.imag
  b[k][i] = Bs.imag

G = Ybus.real
B = Ybus.imag

print(Ybus)
print(G)
print(B)

[[  6.46838347-20.69594776j  -5.22464618+15.64672684j
   -1.24373729 +5.09602092j   0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j        ]
 [ -5.22464618+15.64672684j   9.75228217-30.64866289j
    0.         +0.j          -1.70553032 +5.19737923j
   -1.13596079 +4.77247933j  -1.68614488 +5.1164775j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.         +0.j
    0.         +0.j           0.        

# Objective Function

In [ ]:
def ObjectiveFunction(model):
  Cost = 0.0
  for genid, gendata in gens.items():
    bus = int(gendata[0])
    a = gendata[4]
    b = gendata[5]
    c = gendata[6]

    Cost += c*model.Pgen[bus]**2 + b*model.Pgen[bus] + a

  #return Cost
  return -sum(model.l[i] for i in model.bus)

# Constraints

In [ ]:
def MinGen_P(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  lb = 0
  if keys:
    lb = gens[keys[0]][1]
    
  return model.Pgen[bus] >= lb
  
def MaxGen_P(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  ub = 0
  if keys:
    ub = gens[keys[0]][2]
    
  return model.Pgen[bus] <= ub

def MinGen_Q(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  lb = 0
  if keys:
    lb = gens[keys[0]][3]
    
  return model.Qgen[bus] >= lb
  
def MaxGen_Q(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  ub = 0
  if keys:
    ub = gens[keys[0]][4]
    
  return model.Qgen[bus] <= ub

def MaxFlowLineik(model, line):
  S = lines[line][6]
  i = int(lines[line][0])
  k = int(lines[line][1])
  return model.Pflow[i,k]**2+model.Qflow[i,k]**2 <= S**2

def MaxFlowLineki(model, line):
  S = lines[line][6]
  i = int(lines[line][1])
  k = int(lines[line][0])
  return model.Pflow[i,k]**2+model.Qflow[i,k]**2 <= S**2

def KirchoffBusesP(model, bus):
  Pik = 0
  Pgbus = model.Pgen[bus]
    
  for linea in model.line:
    i = int(lines[linea][0])
    if i == bus: # gen id is the same as bus id
      j = int(lines[linea][1])
      Pik += model.Pflow[i,j]
      
  for linea in model.line:
    i = int(lines[linea][1])
    if i == bus: # gen id is the same as bus id
      j = int(lines[linea][0])
      Pik += model.Pflow[i,j]
    
  return Pgbus == buses[bus][6]*model.l[bus] + Pik

def KirchoffBusesQ(model, bus):
  Qik = 0
  Qgbus = model.Qgen[bus]
  Qshunt = 0
  for linea in model.line:
    i = int(lines[linea][0])
    if i == bus: # gen id is the same as bus id
      j = int(lines[linea][1])
      Qik += model.Qflow[i,j]
      
  for linea in model.line:
    i = int(lines[linea][1])
    if i == bus: # gen id is the same as bus id
      j = int(lines[linea][0])
      Qik += model.Qflow[i,j]
      
    
  return Qgbus == buses[bus][7]*model.l[bus] + Qik + Qshunt
  
# Lines equations
def PflowEq1(model, linea):
  i = int(lines[linea][0])
  j = int(lines[linea][1])
  return model.Pflow[i, j] == (-G[i-1][j-1] + g[i-1][j-1])*model.V[i]**2 + model.V[i]*model.V[j]*(G[i-1][j-1]*cos(model.theta[i]-model.theta[j]) + B[i-1][j-1]*sin(model.theta[i]-model.theta[j]))

def PflowEq2(model, linea):
  i = int(lines[linea][1])
  j = int(lines[linea][0])
  return model.Pflow[i, j] == (-G[i-1][j-1] + g[i-1][j-1])*model.V[i]**2 + model.V[i]*model.V[j]*(G[i-1][j-1]*cos(model.theta[i]-model.theta[j]) + B[i-1][j-1]*sin(model.theta[i]-model.theta[j]))

def QflowEq1(model, linea):
  i = int(lines[linea][0])
  j = int(lines[linea][1])
  return model.Qflow[i, j] == (B[i-1][j-1] - b[i-1][j-1])*model.V[i]**2 + model.V[i]*model.V[j]*(-B[i-1][j-1]*cos(model.theta[i]-model.theta[j]) + G[i-1][j-1]*sin(model.theta[i]-model.theta[j]))

def QflowEq2(model, linea):
  i = int(lines[linea][1])
  j = int(lines[linea][0])
  return model.Qflow[i, j] == (B[i-1][j-1] - b[i-1][j-1])*model.V[i]**2 + model.V[i]*model.V[j]*(-B[i-1][j-1]*cos(model.theta[i]-model.theta[j]) + G[i-1][j-1]*sin(model.theta[i]-model.theta[j]))


# Finally, solve

In [ ]:
model = ConcreteModel()

model.bus = Set(initialize = buses.keys())
model.line = Set(initialize = lines.keys())
model.gen = Set(initialize = gens.keys())

# Create variables
model.Pgen = Var(model.bus, initialize = 0)
model.Qgen = Var(model.bus, initialize = 0)
model.V = Var(model.bus, initialize = 0.0, bounds = (0.95, 1.05), within = NonNegativeReals)
model.theta = Var(model.bus, initialize = 0.0, bounds = (-pi, pi))
model.l = Var(model.bus, initialize = 1.0, bounds = (0.0, 1.0))

# Line flows
model.Pflow = Var(model.bus, model.bus)
model.Qflow = Var(model.bus, model.bus)

model.obj = Objective(rule = ObjectiveFunction, sense = minimize)

model.c0 = Constraint(expr = model.theta[1] == 0)

model.c1 = Constraint(model.bus, rule = KirchoffBusesP)
model.c2 = Constraint(model.bus, rule = KirchoffBusesQ)

model.c3 = Constraint(model.bus, rule = MaxGen_P)
model.c4 = Constraint(model.bus, rule = MinGen_P)
model.c5 = Constraint(model.bus, rule = MaxGen_Q)
model.c6 = Constraint(model.bus, rule = MinGen_Q)


model.c7 = Constraint(model.line, rule = MaxFlowLineik)
model.c8 = Constraint(model.line, rule = MaxFlowLineki)

model.c9 = Constraint(model.line, rule = PflowEq1)
model.c10 = Constraint(model.line, rule = PflowEq2)
model.c11 = Constraint(model.line, rule = QflowEq1)
model.c12 = Constraint(model.line, rule = QflowEq2)


# Solve

In [ ]:
solver = SolverFactory('ipopt', executable='/content/ipopt')
results = solver.solve(model)
print(results.solver.termination_condition)

optimal


In [ ]:
PrintOPFACResults(model, buses, lines, gens, [])

BusID	V	th	Pg	Qg	l	Pl	Ql	Qshunt

1	1.0500	-0.0000	8.0000	0.0000	1.0000	0.0000	0.0000	0.0000
2	1.0037	-0.4575	12.0000	16.7807	1.0000	21.7000	12.7000	0.0000
3	1.0500	0.0187	6.9861	0.0000	1.0000	2.4000	1.2000	0.0000
4	1.0500	-0.1499	5.8648	3.6656	1.0000	7.6000	1.6000	0.0000
5	1.0500	-0.5677	12.0000	4.4097	0.1417	13.3455	2.6918	0.0000
6	1.0500	-0.1574	7.6912	3.3551	1.0000	0.0000	0.0000	0.0000
7	0.9500	-0.4049	0.0000	0.0000	0.0861	1.9638	0.9388	0.0000
8	0.9500	-0.2109	0.0000	0.0000	0.0669	2.0068	2.0068	0.0000
9	1.0040	-0.2355	0.0000	-0.0000	1.0000	0.0000	0.0000	0.0000
10	0.9827	-0.2790	0.0000	-0.0000	0.0543	0.3147	0.1085	0.0000
11	1.0040	-0.2355	0.0000	-0.0000	1.0000	0.0000	0.0000	0.0000
12	0.9844	-0.2693	0.0000	0.0000	0.0000	0.0000	0.0000	0.0000
13	0.9844	-0.2693	0.0000	-0.0000	1.0000	0.0000	0.0000	0.0000
14	0.9500	-0.2940	0.0000	0.0000	0.0529	0.1693	0.0847	0.0000
15	0.9644	-0.2880	0.0000	-0.0000	0.0000	0.0000	0.0000	0.0000
16	0.9500	-0.3003	0.0000	-0.0000	0.0807	0.2824	0.1452	0.0000
17	0.

In [ ]:
def PrintOPFACResults(model, buses, lineas, gens, shunts):

	nb = len(buses)
	nl = len(lineas)
	ng = len(gens)
	ns = len(shunts)

	print('BusID	V	th	Pg	Qg	l	Pl	Ql	Qshunt\n')
	l = {}
	for i in model.bus:
		Qshunt = 0
		Pg = abs(model.Pgen[i]())
		Qg = model.Qgen[i]()
		
		print("{0:.0f}	{1:.4f}	{2:.4f}	{3:.4f}	{4:.4f}	{5:.4f}	{6:.4f}	{7:.4f}	{8:.4f}".format(i,model.V[i](),model.theta[i](),Pg,Qg, model.l[i](), buses[i][6]*model.l[i](), buses[i][7]*model.l[i](), 0.0))

	Pgtotal = sum(model.Pgen[i]() for i in model.bus)
	Qgtotal = sum(model.Qgen[i]() for i in model.bus)

	Ploadtotal = sum(buses[i][6] for i in model.bus)
	Qloadtotal = sum(buses[i][7] for i in model.bus)

	print("\n")
	print("TOTAL			{0:.4f}	{1:.4f}		{2:.4f}	{3:.4f}".format(Pgtotal, Qgtotal, Ploadtotal, Qloadtotal))
	print("\n\n")

	print("Busi	Busk	Pik	Pki	Qik	Qki")
	Pik = np.zeros((nb,nb))
	Pki = np.zeros((nb,nb))
	Qik = np.zeros((nb,nb))
	Qki = np.zeros((nb,nb))

	for l in model.line:
		i = lineas[l][0]
		j = lineas[l][1]
		print("{0:.0f}	{1:.0f}	{2:.4f}	{3:.4f}	{4:.4f}	{5:.4f}".format(i,j,model.Pflow[i,j](),model.Pflow[j,i](),model.Qflow[i,j](),model.Qflow[j,i]()))
		
	Ploss = 0
	Qloss = 0
	for l in model.line:
		i = lineas[l][0]
		j = lineas[l][1]
		Ploss += model.Pflow[i,j]() + model.Pflow[j,i]()
		Qloss += model.Qflow[i,j]() + model.Qflow[j,i]()


	Pl_supplied = sum(buses[i][6]*model.l[i]() for i in model.bus)
	Pl_total = sum(buses[i][6] for i in model.bus)
	perc_supplied = (Pl_supplied/Pl_total)*100
	
	print("\n")
	print("Total Ploss: {0:.4f}\nTotal Qloss: {1:.4f}".format(Ploss,Qloss))
	print("Total Load Supplied: {0:.4f}%".format(perc_supplied))